<a href="https://colab.research.google.com/github/dcolinmorgan/LTCOPDsex/blob/master/gpuMilipeed_%26_ltcopd_sex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os,gc
import s3fs
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from psutil import *
from scipy.stats import zscore

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [37]:
!pip install cupy-cuda101==7.4.0
import cupy as cp
# print(cp.__version__)
!pip freeze | grep cupy

cupy-cuda101==7.4.0


In [38]:
# !rm -rf netZooPy

In [39]:
# !git clone https://github.com/netZoo/netZooPy.git
!git clone --single-branch --branch milipeed https://github.com/dcolinmorgan/netZooPy.git
os.chdir('netZooPy')
!pip install -e ./

Cloning into 'netZooPy'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 3335 (delta 70), reused 69 (delta 58), pack-reused 3231
Receiving objects: 100% (3335/3335), 192.03 MiB | 20.44 MiB/s, done.
Resolving deltas: 100% (1411/1411), done.
Checking out files: 100% (358/358), done.
Obtaining file:///content/netZooPy
     |████████████████████████████████| 3.2MB 3.0MB/s 
  Running setup.py develop for netZooPy


In [40]:
# from importlib import reload  
# reload(netZooPy)

In [41]:
import netZooPy
from netZooPy.panda import Panda
from netZooPy.lioness import Lioness
from netZooPy.milipeed import Milipeed
os.chdir('..')

In [ ]:
# !pip install GEOparse

In [ ]:
# https://geoparse.readthedocs.io/en/latest/Analyse_hsa-miR-124a-3p_transfection_time-course.html#
# import GEOparse
# gsem = GEOparse.get_GEO(geo="GSE92511", destdir="./")
# gsem = GEOparse.get_GEO(filepath="GSE92511_family.soft.gz")

##expression
# gsee = GEOparse.get_GEO(geo="GSE76925", destdir="./")
# gsee = GEOparse.get_GEO(filepath="GSE76925_family.soft.gz")

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
# illumina_labels=pd.read_csv('drive/My Drive/Colab Notebooks/illumina_labels.txt',sep=';',header=0)
labels=pd.read_csv('drive/My Drive/Colab Notebooks dcm/v14bexp.txt',sep='\t',header=0)

In [ ]:
# expression = gsee.pivot_samples('VALUE')
# expression.index=illumina_labels['Gene']
# expression=expression[expression.index.notnull()]
# expression.columns=labels.columns[1:]

In [ ]:
# LCL_ppi=pd.read_csv('https://granddb.s3.amazonaws.com/gpuPANDA/ppi2015_freezeCellLine.txt',sep='\t',header=None)
# LCL_ppi.to_csv('ppi_complete.txt',sep='\t',index=False)
# # expression.to_csv('GPL10558.txt',sep='\t',index=True,header=True)
# # methylation.to_csv('GSE76925.txt',sep='\t',index=True,header=True)
# # # LCL_motif=pd.read_csv('https://granddb.s3.amazonaws.com/gpuPANDA/Hugo_motifCellLine_reduced.txt',sep='\t',header=None)
# # # LCL_motif.to_csv('Hugo_motifCellLine_reduced.txt',sep='\t',index=False)
# # coding_LCL_motif=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/motifs/Hugo_motifCellLine.txt',sep='\t',header=None)
# # coding_LCL_motif[coding_LCL_motif[2]!=0].to_csv('one_coding_LCL_motif.txt',sep='\t',index=False,header=False)
# milipeed_test_motif=pd.read_csv('drive/My Drive/Colab Notebooks dcm/milipeed_test_tr.txt',sep='\t',header=0,index_col=0)
# milipeed_test_motif.columns = milipeed_test_motif.columns.str.replace("-", ".")
# milipeed_test_motif.to_csv('milipeed_test_motif.txt',sep='\t',index=True,header=True)


In [ ]:
# tmp_expdata = expression#pd.read_csv(expression_file,sep='\t',header=0,nrows=0)
# tmp_betadata = milipeed_test_motif#pd.read_csv(methylation_file,sep='\t',header=0,nrows=0)
# b_subj=tmp_betadata.columns[(tmp_betadata.columns).isin(tmp_expdata)].dropna().tolist()
# e_subj=tmp_expdata.columns[(tmp_expdata.columns).isin(tmp_betadata)].dropna().tolist()
# # tmp_betadata2[pd.DataFrame(tmp_betadata2).isin(tmp_expdata2),]
# print('Number of subjects:', len(b_subj))



In [ ]:
# # milipeed_obj2=Milipeed('GPL10558.txt','milipeed_test_motif.txt','ppi_complete.txt',

# milipeed_obj=Milipeed('drive/My Drive/Colab Notebooks dcm/gene_exp_v14b.txt','milipeed_test_motif.txt','ppi_complete.txt',
#     computing='gpu',save_dir='milipeed_output/',
#     save_fmt='npy',map_file='netZooPy/tests/milipeed/MotifPrior_CGmap.txt')
# # milipeed_obj.save_milipeed_results(file='LTCOPDmilipeed_02232020.txt')



In [ ]:
# !zip -r milipeed_output.zip milipeed_output

In [ ]:
# !cp -R milipeed_output.zip "/content/drive/My Drive/"

In [ ]:
# from google.colab import files
# files.download('milipeed_output.zip')

In [ ]:
!pip install GEOparse

In [ ]:
import GEOparse
gse = GEOparse.get_GEO(geo="GSE76925", destdir="./")

In [7]:
# illumina_labels=pd.read_csv('drive/My Drive/Colab Notebooks/illumina_labels.txt',sep=';',header=0)
illumina_labels=pd.read_csv('drive/My Drive/Colab Notebooks dcm/GPL10558_illumina_prb_loc.txt',sep='\t',index_col=0,header=None,names=['loc','symbol'])

In [8]:
gpl = gse.gpls[next(iter(gse.gpls))]
data = gse.pivot_samples(values="VALUE")#, gpl=gpl, annotation_column="Symbol")
data.sort_index

<bound method DataFrame.sort_index of name          GSM2040792  GSM2040793  ...  GSM2040941  GSM2040942
ID_REF                                ...                        
ILMN_1343291   14.716834   14.638305  ...   14.899412   14.854133
ILMN_1343295   11.324115   11.701110  ...   12.050576   12.018367
ILMN_1651209    3.001078    3.129692  ...    3.657480    3.009667
ILMN_1651221    3.207901    4.586259  ...    4.358128    3.009667
ILMN_1651228   11.775397   10.800366  ...   11.595252   12.374196
...                  ...         ...  ...         ...         ...
ILMN_3311145    3.101326    2.908517  ...    4.617594    4.575824
ILMN_3311155    3.425322    3.659397  ...    3.420438    3.341996
ILMN_3311165    2.810761    3.593618  ...    3.950109    3.564369
ILMN_3311180    3.515478    3.107415  ...    4.167176    3.334542
ILMN_3311190    3.445564    4.075027  ...    2.995176    4.263345

[32831 rows x 151 columns]>

In [9]:
pheno=gse.phenotype_data.rename(columns={'characteristics_ch1.0.age': 'age',"characteristics_ch1.1.Sex":"sex","characteristics_ch1.3.bmi":"bmi","characteristics_ch1.6.fev1.pp":"fev1","characteristics_ch1.4.packyears":"PY"})
pheno[['age',"sex","bmi","fev1","PY"]].to_csv('pheno_data.txt',sep='\t',index=True,header=True)

create M and F subpopulations for case and control

In [10]:
data2=data.merge(illumina_labels['symbol'],left_index=True,right_index=True)
data2.index=data2['symbol']
del data2['symbol']
data2.columns=[gse.phenotype_data['characteristics_ch1.1.Sex']+gse.phenotype_data['characteristics_ch1.5.copd']]
data2.loc[:,'Mcase'].to_csv('Mcase.txt',sep='\t',index=True,header=False)
data2.loc[:,'Mcont'].to_csv('Mcont.txt',sep='\t',index=True,header=False)

data3=data.merge(illumina_labels,left_index=True,right_index=True)
data3.index=data3['symbol']
data3=data3[data3['loc']!='Y']
del data3['symbol'], data3['loc']
data3.columns=[gse.phenotype_data['characteristics_ch1.1.Sex']+gse.phenotype_data['characteristics_ch1.5.copd']]
data3.loc[:,'Fcase'].to_csv('Fcase.txt',sep='\t',index=True,header=False)
data3.loc[:,'Fcont'].to_csv('Fcont.txt',sep='\t',index=True,header=False)


data4=data.merge(illumina_labels,left_index=True,right_index=True)
data4.index=data4['symbol']
# data4=data4[data4['loc']!='Y']
del data4['symbol'], data4['loc']
data4.columns=[gse.phenotype_data['characteristics_ch1.1.Sex']+gse.phenotype_data['characteristics_ch1.5.copd']]
data4.loc[:,'Fcase'].to_csv('FcaseFULL.txt',sep='\t',index=True,header=False)
data4.loc[:,'Fcont'].to_csv('FcontFULL.txt',sep='\t',index=True,header=False)

In [12]:
[data2.loc[:,'Mcase'].shape,data2.loc[:,'Mcont'].shape,data3.loc[:,'Fcase'].shape,data3.loc[:,'Fcont'].shape]




[(32831, 52), (32831, 15), (32794, 59), (32794, 25)]

In [43]:
LCL_ppi=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/ppi/ppi_complete.txt',sep='\t',header=None)
LCL_ppi.to_csv('ppi_complete.txt',sep='\t',index=False,header=False)

coding_LCL_motif=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/motifs/Hugo_motifCellLine.txt',sep='\t',header=None)
coding_LCL_motif.to_csv('coding_LCL_motif.txt',sep='\t',index=False,header=False)

motif_data='coding_LCL_motif.txt'
ppi_data='ppi_complete.txt'

In [ ]:
panda_obj1 = Panda('Fcase.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
np.save('drive/My Drive/Panda_Fcase.npy',panda_obj1.export_panda_results)
del panda_obj1
# panda_obj2 = Panda('Mcase.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = True, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
# np.save('drive/My Drive/Panda_Mcase.npy',panda_obj2.export_panda_results)
# del panda_obj2
panda_obj3 = Panda('Fcont.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
np.save('drive/My Drive/Panda_Fcont.npy',panda_obj3.export_panda_results)
del panda_obj3
# panda_obj4 = Panda('Mcont.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = True, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
# np.save('drive/My Drive/Panda_Mcont.npy',panda_obj4.export_panda_results)
# del panda_obj4
panda_obj5 = Panda('FcaseFULL.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
np.save('drive/My Drive/Panda_FcaseFULL.npy',panda_obj5.export_panda_results)
del panda_obj5
panda_obj6 = Panda('FcontFULL.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
np.save('drive/My Drive/Panda_FcontFULL.npy',panda_obj6.export_panda_results)
del panda_obj6


LTCOPD sex analysis (needed more RAM) (Fcase v Fcont)

In [ ]:
Panda_Fcase=np.load('drive/My Drive/Colab Notebooks dcm/Panda_Fcase.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_Mcase=np.load('drive/My Drive/Colab Notebooks dcm/Panda_Mcase.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_Fcont=np.load('drive/My Drive/Colab Notebooks dcm/Panda_Fcont.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_Mcont=np.load('drive/My Drive/Colab Notebooks dcm/Panda_Mcont.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])

Panda_FcaseFULL=np.load('drive/My Drive/Panda_FcaseFULL.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_FcontFULL=np.load('drive/My Drive/Panda_FcontFULL.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])



Panda_Fcont=pd.DataFrame(Panda_Fcont)
Panda_Fcont.columns=['tf','gene','motif','force']
Panda_Fcase=pd.DataFrame(Panda_Fcase)
Panda_Fcase.columns=['tf','gene','motif','force']

Panda_FcontFULL=pd.DataFrame(Panda_FcontFULL)
Panda_FcontFULL.columns=['tf','gene','motif','force']
Panda_FcaseFULL=pd.DataFrame(Panda_FcaseFULL)
Panda_FcaseFULL.columns=['tf','gene','motif','force']

Panda_Mcont=pd.DataFrame(Panda_Mcont)
Panda_Mcont.columns=['tf','gene','motif','force']
Panda_Mcase=pd.DataFrame(Panda_Mcase)
Panda_Mcase.columns=['tf','gene','motif','force']

In [ ]:
# Panda_Fcont.panda_results = pd.DataFrame(Panda_Fcont.export_panda_results, columns=['tf','gene','motif','force'])
subset_panda_results3 = Panda_Fcont.sort_values(by=['force'], ascending=False)
subset_panda_results3['merge']=subset_panda_results3.tf+'-'+subset_panda_results3.gene

# Panda_Fcase.panda_results = pd.DataFrame(Panda_Fcase.export_panda_results, columns=['tf','gene','motif','force'])
subset_panda_results1 = Panda_Fcase.sort_values(by=['force'], ascending=False)
subset_panda_results1['merge']=subset_panda_results1.tf+'-'+subset_panda_results1.gene

www=subset_panda_results3.merge(subset_panda_results1, left_on='merge', right_on='merge')

del [[subset_panda_results3, subset_panda_results1]]
gc.collect()
subset_panda_results3=pd.DataFrame()
subset_panda_results1=pd.DataFrame()

www['diff']=www.force_x-www.force_y
www = www.sort_values(by ='diff' )

wwww=www[np.abs(www['diff'])>2]
del [[www]]
gc.collect()
www=pd.DataFrame()


# len(pd.unique(wwww['tf_x']))
# import seaborn as sns
# plt.figure(figsize=(12, 30))

# g=sns.barplot(x="diff", y="merge", data=wwww)
# g.set(xlim=(-4.5, 4.5))
# wwww.to_csv('female_caVco.txt')
pd.DataFrame([pd.unique(wwww['tf_x']),pd.unique(wwww['gene_x'])]).to_csv('drive/My Drive/unqiue_female_caVco.txt',sep='\t',header=None,index=None)

In [ ]:
del [[Panda_Fcont,Panda_Fcase]]
gc.collect()
Panda_Fcont=pd.DataFrame()
Panda_Fcase=pd.DataFrame()

LTCOPD sex analysis (needed more RAM) (FcaseFULL v FcontFULL)

In [ ]:
# Panda_Fcont.panda_results = pd.DataFrame(Panda_Fcont.export_panda_results, columns=['tf','gene','motif','force'])
subset_panda_results5 = Panda_FcontFULL.sort_values(by=['force'], ascending=False)
subset_panda_results5['merge']=subset_panda_results5.tf+'-'+subset_panda_results5.gene

# Panda_Fcase.panda_results = pd.DataFrame(Panda_Fcase.export_panda_results, columns=['tf','gene','motif','force'])
subset_panda_results6 = Panda_FcaseFULL.sort_values(by=['force'], ascending=False)
subset_panda_results6['merge']=subset_panda_results6.tf+'-'+subset_panda_results6.gene

zzz=subset_panda_results5.merge(subset_panda_results6, left_on='merge', right_on='merge')

del [[subset_panda_results5, subset_panda_results6]]
gc.collect()
subset_panda_results5=pd.DataFrame()
subset_panda_results6=pd.DataFrame()

zzz['diff']=zzz.force_x-zzz.force_y
zzz = zzz.sort_values(by ='diff' )

zzzz=zzz[np.abs(zzz['diff'])>2]
del [[zzz]]
gc.collect()
zzz=pd.DataFrame()


# len(pd.unique(zzzz['tf_x']))
# import seaborn as sns
# plt.figure(figsize=(62, 50))

# g=sns.barplot(x="diff", y="merge", data=zzzz)
# g.set(xlim=(-6.5, 6.5))
# zzzz.to_csv('female_caVco.txt')
# pd.DataFrame([pd.unique(zzzz['tf_x']),pd.unique(zzzz['gene_x'])]).to_csv('drive/My Drive/unqiue_femaleFULL_caVco.txt',sep='\t',header=None,index=None)

In [ ]:
del [[Panda_FcontFULL,Panda_FcaseFULL]]
gc.collect()
Panda_FcontFULL=pd.DataFrame()
Panda_FcaseFULL=pd.DataFrame()

In [ ]:
# panda_obj4.panda_results = pd.DataFrame(panda_obj4.export_panda_results, columns=['tf','gene','motif','force'])
subset_panda_results4 = Panda_Mcase.sort_values(by=['force'], ascending=False)
subset_panda_results4['merge']=subset_panda_results4.tf+'-'+subset_panda_results4.gene

# panda_obj2.panda_results = pd.DataFrame(Panda_Mcont.export_panda_results, columns=['tf','gene','motif','force'])
subset_panda_results2 = Panda_Mcont.sort_values(by=['force'], ascending=False)
subset_panda_results2['merge']=subset_panda_results2.tf+'-'+subset_panda_results2.gene

ww=subset_panda_results4.merge(subset_panda_results2, left_on='merge', right_on='merge')

del [[subset_panda_results4, subset_panda_results2]]
gc.collect()
subset_panda_results4=pd.DataFrame()
subset_panda_results2=pd.DataFrame()

ww['diff']=ww.force_x-ww.force_y
ww = ww.sort_values(by ='diff' )
pppp=ww[np.abs(ww['diff'])>2]

del [[ww]]
gc.collect()
ww=pd.DataFrame()

# zzzz
# len(pd.unique(zzzz['gene_x']))
# import seaborn as sns
# plt.figure(figsize=(12, 30))

# g=sns.barplot(x="diff", y="merge", data=zzzz)
# g.set(xlim=(-4.5, 4.5))
# zzzz.to_csv('male_caVco.txt')
# [pd.unique(zzzz['tf_x']),pd.unique(zzzz['gene_x'])]
# pd.DataFrame([pd.unique(zzzz['tf_x']),pd.unique(zzzz['gene_x'])]).to_csv('drive/My Drive/unqiue_male_caVco.txt',sep='\t',header=None,index=None)

In [ ]:
del [[Panda_Mcont,Panda_Mcase]]
gc.collect()
Panda_Mcont=pd.DataFrame()
Panda_Mcase=pd.DataFrame()

Female V Female Full

In [ ]:
sexdiff=wwww.merge(zzzz,left_on='merge', right_on='merge')

plt.figure(figsize=(6, 15))
sexdiff['sexdiff']=sexdiff['diff_x']-sexdiff['diff_y']
sexdiff=sexdiff.sort_values(by=['sexdiff'])
sns.barplot(data=sexdiff,x='sexdiff',y='merge')
# g=sns.barplot(x="diff", y="merge", data=wwww)
# g.set(xlim=(-4.5, 4.5))

Female FULL V Male

In [ ]:
sexdiff=zzzz.merge(pppp,left_on='merge', right_on='merge')

plt.figure(figsize=(6, 15))
sexdiff['sexdiff']=sexdiff['diff_x']-sexdiff['diff_y']
sexdiff=sexdiff.sort_values(by=['sexdiff'])
sns.barplot(data=sexdiff,x='sexdiff',y='merge')
# g=sns.barplot(x="diff", y="merge", data=wwww)
# g.set(xlim=(-4.5, 4.5))

Female V Male

In [ ]:
sexdiff=wwww.merge(pppp,left_on='merge', right_on='merge')

plt.figure(figsize=(6, 15))
sexdiff['sexdiff']=sexdiff['diff_x']-sexdiff['diff_y']
sexdiff=sexdiff.sort_values(by=['sexdiff'])
sns.barplot(data=sexdiff,x='sexdiff',y='merge')
# g=sns.barplot(x="diff", y="merge", data=wwww)
# g.set(xlim=(-4.5, 4.5))

In [ ]:
from scipy.stats import zscore

In [ ]:
PZ=pd.DataFrame()
PZ['TF']=Panda_FcontFULL['tf']
PZ['gene']=Panda_FcontFULL['gene']
Panda_FcontFULL['force']=Panda_FcontFULL['force'].astype('float')
numeric_cols = Panda_FcontFULL.select_dtypes(include=[np.number]).columns
PZ['FcoF']=Panda_FcontFULL[numeric_cols].apply(zscore)

Panda_FcaseFULL['force']=Panda_FcaseFULL['force'].astype('float')
numeric_cols = Panda_FcaseFULL.select_dtypes(include=[np.number]).columns
PZ['FcaF']=Panda_FcaseFULL[numeric_cols].apply(zscore)

Panda_Mcont['force']=Panda_Mcont['force'].astype('float')
numeric_cols = Panda_Mcont.select_dtypes(include=[np.number]).columns
PZ['Mco']=Panda_Mcont[numeric_cols].apply(zscore)

Panda_Mcase['force']=Panda_Mcase['force'].astype('float')
numeric_cols = Panda_Mcase.select_dtypes(include=[np.number]).columns
PZ['Mca']=Panda_Mcase[numeric_cols].apply(zscore)

In [ ]:
PZZ=pd.DataFrame()
PZZ['TF']=Panda_Fcont['tf']
PZZ['gene']=Panda_Fcont['gene']
Panda_Fcont['force']=Panda_Fcont['force'].astype('float')
numeric_cols = Panda_Fcont.select_dtypes(include=[np.number]).columns
PZZ['Fco']=Panda_Fcont[numeric_cols].apply(zscore)

Panda_Fcase['force']=Panda_Fcase['force'].astype('float')
numeric_cols = Panda_Fcase.select_dtypes(include=[np.number]).columns
PZZ['Fca']=Panda_Fcase[numeric_cols].apply(zscore)

In [ ]:
PZZZ=PZ.merge(PZZ,how='outer',left_on=['TF','gene'],right_on=['TF','gene'])

In [ ]:
PZZZ

In [ ]:
PZZZ.dropna(how='any')

In [ ]:
PZZZ.to_csv('drive/My Drive/Panda_zscore_6sex.txt',sep='\t',index=True,header=False)

In [ ]:
pd.read_csv('drive/My Drive/Panda_zscore_6sex.txt',sep='\t',index=True,header=False)

In [ ]:
PZ=PZZZ
PZ['FFdiff']=PZ['FcoF']-PZ['FcaF']
PZ['Fdiff']=PZ['Fco']-PZ['Fca']
PZ['Mdiff']=PZ['Mco']-PZ['Mca']
PZ

In [ ]:
a=plt.hist(PZ['FFdiff'],log=True,alpha=.5,bins=100)
# a=plt.hist(PZ['Fdiff'],log=True,alpha=.5,bins=100)
a=plt.hist(PZ['Mdiff'],log=True,alpha=.5,bins=100)

In [ ]:
sum((pd.unique(PZ[np.abs(PZ['FFdiff'])>2]['TF']))==(pd.unique(PZ[np.abs(PZ['Fdiff'])>2]['TF'])))

In [ ]:
pd.unique(PZ[(PZ['FFdiff'])>2]['TF'])

In [ ]:
pd.unique(PZ[(PZ['FFdiff'])<-2]['TF'])

In [ ]:
(pd.unique(PZ[(PZ['Mdiff'])>2]['TF'])).shape

In [ ]:
(pd.unique(PZ[(PZ['Mdiff'])<-2]['TF']))

In [ ]:
pd.unique(PZ[(PZ['FFdiff'])>2.5]['gene'])#)==(pd.unique(PZ[np.abs(PZ['Fdiff'])>2.5]['gene'])))

In [ ]:
pd.unique(PZ[(PZ['FFdiff'])<-2.5]['gene'])

In [ ]:
(pd.unique(PZ[(PZ['Mdiff'])>2.5]['gene']))

In [ ]:
(pd.unique(PZ[(PZ['Mdiff'])<-2.5]['gene']))

##negative control
create 4 randomly chosen subsets to test difference of differences between panda nets







In [7]:
!pip install GEOparse
import GEOparse
gse = GEOparse.get_GEO(geo="GSE76925", destdir="./")
illumina_labels=pd.read_csv('drive/My Drive/Colab Notebooks dcm/GPL10558_illumina_prb_loc.txt',sep='\t',index_col=0,header=None,names=['loc','symbol'])

gpl = gse.gpls[next(iter(gse.gpls))]
data = gse.pivot_samples(values="VALUE")#, gpl=gpl, annotation_column="Symbol")
data.sort_index

LCL_ppi=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/ppi/ppi_complete.txt',sep='\t',header=None)
LCL_ppi.to_csv('ppi_complete.txt',sep='\t',index=False,header=False)

coding_LCL_motif=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/motifs/Hugo_motifCellLine.txt',sep='\t',header=None)
coding_LCL_motif.to_csv('coding_LCL_motif.txt',sep='\t',index=False,header=False)

motif_data='coding_LCL_motif.txt'
ppi_data='ppi_complete.txt'

# plt.hist(nullA.values.flatten(),alpha=.5,log=True)
# plt.hist(nullB.values.flatten(),alpha=.5,log=True)
# plt.hist(nullC.values.flatten(),alpha=.5,log=True)
# plt.hist(nullD.values.flatten(),alpha=.5,log=True)
# plt.xlabel('test')


In [8]:
null=data.merge(illumina_labels,left_index=True,right_index=True)
null.index=null['symbol']
del null['loc'], null['symbol']

function to run negative control 1000x

In [43]:
def negCont(nullA,nullB,i):
  panda_nullA = Panda(nullA, motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
  # np.save('drive/My Drive/Panda_nullA.npy',panda_nullA.export_panda_results)
  # @blockPrinting
  panda_nullB = Panda(nullB, motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
  # np.save('drive/My Drive/Panda_nullB.npy',panda_nullB.export_panda_results)

  subset_panda_results3 = panda_nullA.export_panda_results.sort_values(by=['force'], ascending=False)
  subset_panda_results1 = panda_nullB.export_panda_results.sort_values(by=['force'], ascending=False)
  del subset_panda_results3['motif'],subset_panda_results1['motif']
  del panda_nullA, panda_nullB

  ##aggregate TF link sum and zscore
  TFsum_3=subset_panda_results3.groupby(['tf']).agg({'force':sum})
  TFsum_1=subset_panda_results1.groupby(['tf']).agg({'force':sum})
  TFsum_1['force']=zscore(TFsum_1['force'])
  TFsum_3['force']=zscore(TFsum_3['force'])

  ##aggregate gene link sum and zscore
  geneSum_3=subset_panda_results3.groupby(['gene']).agg({'force':sum})
  geneSum_1=subset_panda_results1.groupby(['gene']).agg({'force':sum})
  geneSum_3['force']=zscore(geneSum_3['force'])
  geneSum_1['force']=zscore(geneSum_1['force'])

  ##merge
  nnn=TFsum_3.merge(TFsum_1, left_on=['tf'], right_on=['tf'])
  nnnn=geneSum_3.merge(geneSum_1, left_on=['gene'], right_on=['gene'])

  del [[subset_panda_results3, subset_panda_results1]]
  gc.collect()
  subset_panda_results3=pd.DataFrame()
  subset_panda_results1=pd.DataFrame()

  ##diff
  nnn['diff']=nnn.force_x-nnn.force_y
  # nnn = nnn.sort_values(by ='diff' )
  del nnn['force_x'], nnn['force_y']
  nnn=nnn.sort_index(axis=0)
  nnn=nnn.T
  nnnn['diff']=nnnn.force_x-nnnn.force_y
  # nnnn = nnnn.sort_values(by ='diff' )
  del nnnn['force_x'], nnnn['force_y']
  nnnn=nnnn.sort_index(axis=0)
  nnnn=nnnn.T
  if i==1:
    nnn.to_csv('drive/My Drive/TF_zscore_rand_1000.txt',sep='\t',header=True,index=None)
    nnnn.to_csv('drive/My Drive/gene_zscore_rand_1000.txt',sep='\t',header=True,index=None)
  elif i!=1:
    nnn.to_csv('drive/My Drive/TF_zscore_rand_1000.txt',sep='\t',header=None,index=None,mode='a')
    nnnn.to_csv('drive/My Drive/gene_zscore_rand_1000.txt',sep='\t',header=None,index=None,mode='a')

for i in range(1,1000):
  ## four random group exact same size as LTCOPD cases and controls
  null=data.merge(illumina_labels,left_index=True,right_index=True)
  null.index=null['symbol']
  del null['loc'], null['symbol']
  j=np.random.choice(151, size=(67, ),replace=False) ##67 total males
  nullA=null.iloc[:,j.tolist()] ##reduce to random 68
  null2=null.drop(nullA.columns,axis=1) ## create second group without these 68

  k=np.random.choice(j,15,replace=False) ##15 male controls
  nullB=null.iloc[:,k.tolist()] ##take 15 of 68 for male control random
  nullA=nullA.drop(nullB.columns,axis=1) ## remove these 15 from male cases

  l=np.random.choice(null2.shape[1], size=(84, ),replace=False) ##84 total females
  nullC=null2.iloc[:,l.tolist()] ##subset leftover from first null to 85
  m=np.random.choice(l[l<85],25,replace=False) ##25 controls
  nullD=nullC.iloc[:,m.tolist()] ##take 25 for female random control
  nullC=nullC.drop(nullD.columns,axis=1)

  negCont(nullA,nullB,i)
  negCont(nullC,nullD,i+1)

  print('saved iteration='+str(i))

KeyboardInterrupt: ignored